# Problem

Please write a simple compiler – program, which translates one quantum circuit into another, using a restricted set of gates.

You need to consider just the basic gates for the input circuit, such as (I, H, X, Y, Z, RX, RY, RZ, CNOT, CZ).

The output circuit should consist only from the following gates: RX, RZ, CZ.

For example, a Hadamard gate after compilation looks like this:

RX(pi/2)
RZ(pi/2)

Analyze what’s the overhead of the compiled program compared to the original one and propose how to improve it.


# Idea

As a naive approach, we convert each gate into the RX, RZ, CZ equivalent (which can be verified using matrix operations).

$$ I = R_x(0) \\ $$

$$ H = R_x(\pi/2)R_z(\pi/2)\\ $$ 

$$X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} = -iR_x(\pi) \equiv R_x(\pi) ^{[1]}\\ $$

$$Y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix} = iXZ = R_x(\pi)*R_z(\pi) ^{[2]}\ (Y \equiv R_y(\pi)) ^{[1]}\\ $$

$$Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix} = -iR_z(\pi)  \equiv R_z(\pi) ^{[1]}\\ $$

$$R_x(\theta) = \begin{bmatrix} \cos\frac{\theta}{2} & -i\sin\frac{\theta}{2} \\ -i\sin\frac{\theta}{2} & \cos\frac{\theta}{2} \end{bmatrix} = R_x(\theta)\\ $$

$$R_y(\theta) = \begin{bmatrix} \cos\frac{\theta}{2} & -\sin\frac{\theta}{2} \\ \sin\frac{\theta}{2} & \cos\frac{\theta}{2} \end{bmatrix} = R_x(\theta)*R_z(\theta)\\ $$

$$R_z(\theta) = \begin{bmatrix} e^{-i\theta/2} & 0 \\ 0 & e^{i\theta/2} \end{bmatrix} = R_z(\theta)\\ $$

$$CNOT = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix} = (I \otimes H)*(CZ)*(I \otimes H) = (I \otimes R_x(\pi/2)R_z(\pi/2))*(CZ)*(I \otimes R_x(\pi/2)R_z(\pi/2))\\ $$

$$CZ = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1 \end{bmatrix} = CZ\\ $$

In [25]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import HGate, RYGate

In [37]:
x=1
qc = QuantumCircuit(1)
qc.rx(x,0)
qc.decompose().draw()

┌────────┐
q_0: ┤ R(1,0) ├
     └────────┘

In [44]:
tr = RYGate(13)
tr.decompositions[0].draw()

┌────────────┐
q_0: ┤ R(13,pi/2) ├
     └────────────┘

In [6]:
qc = QuantumCircuit(2)
qc.h(1)
qc.x(0)
qc.measure_all()

In [8]:
qc.draw()

┌───┐ ░ ┌─┐   
   q_0: ┤ X ├─░─┤M├───
        ├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├─░──╫─┤M├
        └───┘ ░  ║ └╥┘
meas_0: ═════════╩══╬═
                    ║ 
meas_1: ════════════╩═

In [24]:
for instr in qc.data:
    print(type(instr[0]))

<class 'qiskit.circuit.library.standard_gates.h.HGate'>
<class 'qiskit.circuit.library.standard_gates.x.XGate'>
<class 'qiskit.circuit.barrier.Barrier'>
<class 'qiskit.circuit.measure.Measure'>
<class 'qiskit.circuit.measure.Measure'>


In [14]:
HGate

qiskit.circuit.library.standard_gates.h.HGate

# Footnotes

1. This paper claims that rotation matrices can approximate Pauli Matrices upto a global undetectable phase of $-i$. _Page 5_ https://arxiv.org/pdf/1603.07678.pdf
2. $Y=iXZ$. _Slide 15_, http://www.vcpc.univie.ac.at/~ian/hotlist/qc/talks/bloch-sphere-rotations.pdf

# Further Work

1. **Wild idea #1** How about we generalize this compilation further from a basic set of gates to any general operation, solely using matrix operations. Is there any way to convert _any_ given unitary matrix into a linear combination of other (fixed) unitary matrices of the same size.

2. **Wild Idea #2** How about we merge the idea of task #2 and this task #3 and let the parameter (and count) of $R_x$ and $R_z$  to convert to another single-qubit matrix be decided by gradient descent? The cost function could be the difference squared of each element of the matrix.